In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.optimizers as optimizers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import random
import numpy

tf.keras.backend.clear_session()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
ds = tfds.load("fashion_mnist", split=["train", "test"], shuffle_files=True, as_supervised=True)

In [ ]:
# Creates a list of training and testing
train, test = ds
print(ds)
print(len(train))
print(len(test))

In [ ]:
count = 0
for x, y in tfds.as_numpy(train):
    if count == 0:
        print(x.shape)
        print(y)
        plt.imshow(x)
        break
    count += 1

In [ ]:
train_images = []
train_labels = []
test_images = []
test_labels = []

for x, y in tfds.as_numpy(train):
    x = (x-127.5)/127.5
    train_images.append(x)
    train_labels.append(y)

for x, y in tfds.as_numpy(test):
    x = (x-127.5)/127.5
    test_images.append(x)
    test_labels.append(y)

print(len(train_images))
print(len(train_labels))
print(len(test_images))
print(len(test_labels))

train_images = numpy.asarray(train_images)
test_images = numpy.asarray(test_images)

In [ ]:
for i in range(100):
    plt.subplot(10,10, i+1)
    plt.axis("off")
    plt.imshow(train_images[i], cmap="gray_r")
plt.show()

In [ ]:
def r_discriminator(shape=(28,28,1)):
    model = tf.keras.models.Sequential()
    model.add(layers.Conv2D(128, (3,3), strides=(2,2), padding='same', input_shape=shape))
    model.add(layers.LeakyReLU(alpha=0.2))
    # downsample
    model.add(layers.Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    # classifier
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(1, activation='sigmoid'))
    # compile model
    opt = optimizers.Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model


In [ ]:
def define_generator(latent_dim):
    model = tf.keras.models.Sequential()	
    # foundation for 7x7 image
    # make sure to begin the convolutions at the same dimension the discriminator ends at (128 channels and it had a 7x7 shape)
    n_nodes = 128 * 7 * 7
    model.add(layers.Dense(n_nodes, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((7, 7, 128)))
    # upsample to 14x14
    model.add(layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    # upsample to 28x28
    model.add(layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    # generate
    model.add(layers.Conv2D(1, (7,7), activation='tanh', padding='same'))
    return model

In [ ]:
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = tf.keras.models.Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	opt = optimizers.Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = numpy.random.randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [ ]:
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = numpy.zeros((n_samples, 1))
	return X, y

In [ ]:
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = numpy.random.randint(0, dataset.shape[0], n_samples)
    # select images
    X = dataset[ix]
    
    # generate class labels
    y = numpy.ones((n_samples, 1))
    return X, y

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = numpy.ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
            (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        # save the generator model
    g_model.save('generator.h5')

In [ ]:
latent_dim = 100
# create the discriminator
discriminator = r_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = train_images
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

In [ ]:
from tensorflow.keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot

# create and save a plot of generated images (reversed grayscale)
def show_plot(examples, n):
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
	pyplot.show()

# load model
model = load_model('generator.h5')
# generate images
latent_points = generate_latent_points(100, 100)
# generate images
X = model.predict(latent_points)
# plot the result
show_plot(X, 10)